In [1]:
# 03_recommend_by_cosine_similarity.py

import pandas as pd
from janome.tokenizer import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# --- 準備：前処理用の関数 ---
def get_meaningful_tokens(text, tokenizer, stop_words):
    """文章を単語に分解し、重要な単語リストを返す"""
    tokens = []
    for token in tokenizer.tokenize(text):
        part_of_speech = token.part_of_speech.split(',')[0]
        if part_of_speech in ['名詞', '動詞', '形容詞'] and token.surface not in stop_words:
            tokens.append(token.base_form)
    return tokens

def text_to_bigram_string(text, tokenizer, stop_words):
    """テキストをBi-gramの文字列に変換する"""
    tokens = get_meaningful_tokens(text, tokenizer, stop_words)
    bigrams = [f"{tokens[i]}_{tokens[i+1]}" for i in range(len(tokens) - 1)]
    return ' '.join(bigrams)

# --- メイン処理 ---
if __name__ == '__main__':
    # 前処理済みCSVと分析ツールを準備
    try:
        df = pd.read_csv('/Users/dangararara/lecture/miraisouzou/csv/preprocessed_bigrams.csv')
    except FileNotFoundError:
        print("エラー: 'preprocessed_bigrams.csv' が見つかりません。")
        exit()

    df.dropna(subset=['bigram_corpus'], inplace=True)
    t = Tokenizer()
    stop_words = ['こと', 'もの', 'それ', 'これ', 'さん', 'よう', 'ため', 'こちら',
                   'ところ', '感じ', '利用', '訪問', '注文', '提供', '料理', 'お店', 
                   '雰囲気', '時間', '今回', '非常', '思う', 'いる', 'する', 'なる', 
                   'ある', 'ない', 'くる', 'いく', '良い'
                   ]
    
    # お店ごとの特徴ベクトル（TF-IDF）を作成
    corpus = df['bigram_corpus'].tolist()
    shop_names = df['shop_name'].tolist()
    
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)

    # 推薦ロジック
    
    # ユーザーの曖昧な要望を定義
    user_query = "BBQできるテラスあるお店"
    
    print(f"ユーザーの要望: 「{user_query}」\n")

    # ユーザーの要望を、お店と同じ形式のベクトルに変換
    query_bigrams = text_to_bigram_string(user_query, t, stop_words)
    query_vector = vectorizer.transform([query_bigrams])

    # コサイン類似度を計算
    #    ユーザーのベクトルと、全店舗のベクトル行列との類似度を一度に計算
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix)

    # 結果を整形してランキング表示
    #    計算結果から、各店舗のスコアをリストとして取り出す
    scores = cosine_similarities[0]
    
    # 店舗名とスコアをまとめたDataFrameを作成
    results_df = pd.DataFrame({
        'shop_name': shop_names,
        'score': scores
    })
    
    # スコアが高い順に並べ替え、上位10件を取得
    top_10_recommendations = results_df.sort_values(by='score', ascending=False).head(10)

    print("--- 推薦結果 TOP 10 ---")
    print(top_10_recommendations.to_string(index=False))

ユーザーの要望: 「BBQできるテラスあるお店」

--- 推薦結果 TOP 10 ---
                        shop_name    score
            WOOD DESIGN PARK お台場店 0.054071
                   都立潮風公園バーベキュー広場 0.051103
                      ダブルティー アリアケ 0.032376
THE BBQ BEACH in AQUA CITY ODAIBA 0.027274
                       ロングボード・カフェ 0.022671
                     シズラー アクアシティ店 0.018636
                    都会の農園バーベキュー広場 0.016022
             ディップガーデン テラス 有明ガーデン店 0.012363
                          エフカイビーチ 0.011912
              KING OF THE PIRATES 0.008446
